# Estimate user pp at time of play
**Contributors:** Victor Lin

**Achievement:** Player pp at the time a score was set can be estimated by computing pp on all scores the player set before the play.

**Requirements:**

1. *sql_migration/sql_to_mongo_migration.ipynb*

## Introduction
Except for very recent plays, a player's pp does not reflect their skill at the time a score was set. Therefore, the amount of useable data would increase significantly if an accurate estimate of the user's pp at the time of play can be calculated

In [ ]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
from tqdm import tqdm
import numpy as np
from exploration.config import mongo_inst
from mlpp.data_collection.pp_calculation import fast_pp_hist

In [ ]:
osu_db = mongo_inst['val_random_db']

In [ ]:
user_ids = list(map(lambda c: c['_id'], osu_db['osu_user_stats'].find({}, {})))

In [ ]:
cnt_bonus = []
curr = 1
for i in range(100000):
    curr *= .9994
    cnt_bonus.append(416.6667 * (1 - curr))
cnt_bonus = np.array(cnt_bonus)

## Estimating scores for each Player
fast_pp_history enables fast calculation of the player's estimated pp for a date-sorted list of scores. Score estimation calculations grouped by player.

In [ ]:
for user_id in tqdm(user_ids):
    scores = list(osu_db['osu_scores_high'].find({'user_id': user_id}, {'pp': 1, 'date': 1}).sort('date'))
    score_hist = list(map(lambda x: x['pp'] if x['pp'] else 0, scores))
    pp_hist = np.array(fast_pp_hist(score_hist))
    bonus_pp_hist = pp_hist + cnt_bonus[:len(pp_hist)]

    score_updates = []
    for i in range(len(scores)):
        query = {'_id': scores[i]['_id']}
        update = {
            '$set': {
                'mlpp.est_user_raw_pp': pp_hist[i],
                'mlpp.est_user_pp': bonus_pp_hist[i]
            }
        }
        score_updates.append(UpdateOne(query, update))
    
    if len(score_updates) > 0:
        osu_db['osu_scores_high'].bulk_write(score_updates)

        osu_db['osu_user_stats'].update_one({'_id': user_id}, {
            '$set': {
                'mlpp': {
                    'est_current_raw_pp': pp_hist[-1],
                    'est_current_pp': bonus_pp_hist[-1]
                }
    if len(score_updates) > 0:
        osu_random_db['osu_scores_high'].bulk_write(score_updates)

    osu_random_db['osu_user_stats'].update_one({'_id': user_id}, {
        '$set': {
            'mlpp': {
                'est_current_raw_pp': pp_hist[-1],
                'est_current_pp': bonus_pp_hist[-1]
            }
        }
    })
    else:
        osu_db['osu_user_stats'].remove({'_id': user_id})

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f93d0822-db5a-47ef-9a78-57b8adfbeb20' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>